In [1]:
import json

In [3]:
# Step 1: Load Data
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['tags']

hozana_intents= load_data('sample_intents.json')
hozana_intents

[{'tag': 'greeting',
  'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'How are you?'],
  'responses': ['Hello', 'Hi', 'Hi there', 'Hello how are you?']},
 {'tag': 'thanks',
  'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'],
  'responses': ['Happy to help!',
   'Any time!',
   'My pleasure',
   "You're most welcome!"]}]

In [4]:
# Step 2: Add New tags, Patterns, and Responses
new_intents = [
    {
        "tag": "Order Coffee",
        "patterns": ["I'd like to order a coffee.","Can I get a coffee, please?","I want to buy some coffee.", "A coffee, please"],
        "responses": ["Sure, what type of coffee would you like?","Of course, which size would you prefer?","Great choice! Would you like anything else with your coffee?"]
    },
    {
        "tag": "Customize Order",
        "patterns": ["I'd like my coffee with milk.","Can you make my coffee extra hot?","I want just a black coffee, please","I have a special request for my coffee.","Do you have ice coffee?"],
        "responses": ["Certainly, we can customize your coffee as per your preferences.","Sure, we'll make it just the way you like it.","No problem, please let us know your specific requirements."]
    },
    {
        "tag": "Check Availability",
        "patterns": ["Do you have London Fog available?","Is Coffee Grain in stock?","Are you serving London Fog today?"],
        "responses": ["Yes, Londong Fog is currently available.","Let me check for you... Yes, London Fog is in stock.","I'm sorry, London Fog is not available today. Would you like something else?"]
    },
    {
        "tag": "Place Repeat Order",
        "patterns": ["I want the same coffee I had last time.","Can you repeat my previous order?","Do you remember what I ordered last time?","The same yesterday, please"],
        "responses": ["Sure, we'll prepare your usual order right away.","Of course, we have your previous order saved. Would you like the same today?","Yes, we have your order history on file. Would you like to order the same as before?"]
    },
    {
        "tag": "Check Order Status",
        "patterns": ["Can you tell me the status of my order?","Has my coffee been prepared yet?","When will my order be ready?","I want to check if my coffee is ready "],
        "responses": ["Let me check on that for you... Your order is currently being prepared.","Your coffee is ready and waiting for pickup!","I'm sorry for the delay, your order will be ready in a few minutes."]
    },
    {
        "tag": "Cancel Order",
        "patterns": ["I need to cancel my order.","Can you cancel my coffee?","I changed my mind, I don't want the coffee anymore."],
        "responses": ["Sure, your order has been canceled.","No problem, we've canceled your order as requested.","That's alright, your order has been removed from our system."]
    },
    {
        "tag": "Payment Inquiry",
        "patterns": ["What payment methods do you accept?","Do you take credit cards?","Is cash the only payment option?"],
        "responses": ["We accept cash, credit/debit cards, and mobile payments.","Yes, we accept all major credit cards.","Cash is one of the payment options, but we also accept cards and digital payments."]
    },
    {
        "tag": "Feedback",
        "patterns": ["I'd like to leave some feedback.","Can I give you some suggestions?","Is there a way to provide feedback about my experience?"],
        "responses": ["We appreciate your feedback! Please feel free to share.","Absolutely, we welcome any suggestions you may have.","Yes, we value your input. Please share your thoughts with us."]
    },
    {
        "tag": "Order for Pickup",
        "patterns": ["I'd like to pick up my order.","Is my order ready to pick up?"],
        "responses": ["Sure, your order will be ready for pickup shortly.", "Sure", "Your order will be ready in 2 minutes"]
    },
    {
        "tag": "Order for Delivery",
        "patterns": ["Can you deliver my coffee to my address?","Do you offer delivery services?"],
        "responses": ["We offer delivery services. Please provide your address for delivery.","Yes, we provide delivery services. Please provide your address for delivery."]
    },
    {
        "tag": "Special Offers",
        "patterns": ["Do you have any discounts today?","Are there any promotions for regular customers?","Can I use a coupon for my order?"],
        "responses": ["Yes, we have a special offer on [specific item] today!","As a regular customer, you qualify for our loyalty program benefits.","Sure, please present your coupon at the checkout for a discount."]
    },
    {
        "tag": "Size Options Inquiry",
        "patterns": ["What sizes do you offer for coffee?","Can I get my coffee in different sizes?","Do you have small, medium, and large options?"],
        "responses": ["Yes, we offer small, medium, and large sizes for our coffee.","You can choose from our range of sizes: small, medium, and large.","Certainly, we have a variety of sizes available to suit your preference."]
    },
    {
        "tag": "Ingredients Inquiry",
        "patterns": ["What ingredients do you use in your coffee?","Are your coffee ingredients organic?","Can you tell me about the milk you use?"],
        "responses": ["Our coffee is made with freshly ground beans and high-quality milk.","We use organic ingredients in our coffee preparations.","Our milk is sourced locally and is of the highest quality."]
    },
    {
        "tag": "Recommendation Request",
        "patterns": ["Can you recommend a coffee for me?","What's your most popular coffee?","Do you have any specialty coffees you'd recommend?"],
        "responses": ["Our [specialty coffee] is a customer favorite, I highly recommend it!","Our barista's special blend is always a popular choice.","If you're feeling adventurous, I suggest trying our [specialty coffee], it's a unique blend."]
    },
    {
        "tag": "Allergy Information Inquiry",
        "patterns": ["Do you have allergen information for your coffee?","Are there any allergens I should be aware of in your products?","Can you tell me if your coffee contains nuts/dairy/gluten?"],
        "responses": ["We provide allergen information for our coffee products upon request.","Our coffee products are labeled with allergen information for your convenience.","Please let us know if you have any specific allergen concerns, and we'll be happy to assist you."]
    }
          
]

In [5]:
hozana_intents.extend(new_intents)

In [6]:
hozana_intents

[{'tag': 'greeting',
  'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'How are you?'],
  'responses': ['Hello', 'Hi', 'Hi there', 'Hello how are you?']},
 {'tag': 'thanks',
  'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'],
  'responses': ['Happy to help!',
   'Any time!',
   'My pleasure',
   "You're most welcome!"]},
 {'tag': 'Order Coffee',
  'patterns': ["I'd like to order a coffee.",
   'Can I get a [coffee type], please?',
   'I want to buy some coffee.'],
  'responses': ['Sure, what type of coffee would you like?',
   'Of course, which size would you prefer?',
   'Great choice! Would you like anything else with your coffee?']},
 {'tag': 'Customize Order',
  'patterns': ["I'd like my coffee with [milk/sugar/cream].",
   'Can you make my coffee extra hot/strong?',
   'I have a special request for my coffee.'],
  'responses': ['Certainly, we can customize your coffee as per your preferences.',
   "Sure, we'll make it just the way you like it.",


: 